In [ ]:
import numpy as np
import pandas as pd
import pybaseball as pb

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
class DataFrameEncoder(TransformerMixin):

    def __init__(self):
        """Encode the data.

        Columns of data type object are appended in the list. After 
        appending Each Column of type object are taken dummies and 
        successively removed and two Dataframes are concated again.

        """
    def fit(self, X, y=None):
        self.object_col = []
        for col in X.columns:
            if(X[col].dtype == np.dtype('O')):
                self.object_col.append(col)
        return self

    def transform(self, X, y=None):
        dummy_df = pd.get_dummies(X[self.object_col],drop_first=False)
        X = X.drop(X[self.object_col],axis=1)
        X = pd.concat([dummy_df,X],axis=1)
        return X


In [ ]:
pb.cache.enable()
pd.set_option("display.max_columns", None)

In [ ]:
 pitching = pd.read_csv("data/lahman_1871-2024_csv/Pitching.csv")
 players = pd.read_csv("data/lahman_1871-2024_csv/People.csv")
 batting = pd.read_csv("data/lahman_1871-2024_csv/Batting.csv")
 appearances = pd.read_csv("data/lahman_1871-2024_csv/Appearances.csv")

In [ ]:
pitchData = pb.statcast(start_dt="2024-03-01", end_dt="2025-10-31")

In [ ]:
# Look at pitch movement columns
mainDF = pitchData[['game_date','player_name','events','description','p_throws','hit_location','bb_type','balls','strikes','pitch_type', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','on_3b','on_2b','on_1b','outs_when_up','inning','inning_topbot','release_pos_y','at_bat_number','pitch_number','pitch_name','bat_score','fld_score','post_bat_score','post_fld_score', 'delta_home_win_exp','delta_run_exp','bat_speed','swing_length','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle', 'batter','pitcher']]

In [ ]:
ver = mainDF[mainDF['player_name'] == 'Verlander, Justin'].copy()

In [ ]:
ver["events"].value_counts()

In [ ]:
ver["pitch_success"] = np.where(
    (
        ver["events"].isin([
            'field_out', 'strikeout', 'field_error', 'fielders_choice',
            'double_play', 'grounded_into_double_play', 'force_out'
        ])
    ) |
    (
        ver["description"].isin(['called_strike', 'swinging_strike'])
    ) |
    (
        ver["description"].isin(['foul']) & (ver['strikes'] != 2)
    ),
    True,
    False
)

In [ ]:
conditions = [
    ver["events"].isin(['double_play', 'grounded_into_double_play']),
    ver["events"].isin(['strikeout']), 
    ver["events"].isin(['field_out']),
    ver["description"].isin(['swinging_strike', 'called_strike']),
    ver["description"].isin(['foul', 'foul_tip']),
    ver["description"].isin(['ball']),
    ver["description"].isin(['hit_by_pitch']),
    ver["events"].isin(['single']),  # bad outcomes
    ver["events"].isin(['double']),  # bad outcomes
    ver["events"].isin(['triple']),  # bad outcomes
    ver["events"].isin(['home_run']),  # bad outcomes
]

values = [1.00, 0.95, 0.90, 0.75, 0.70, 0.50, 0.25, 0.25, 0.20, 0.15, 0]

ver["pitch_outcome_score"] = np.select(conditions, values, default=np.nan)

In [ ]:
np.select(conditions, values, default=np.nan)

In [ ]:
ver["on_3b"] = ver["on_3b"].fillna(0)
ver["on_2b"] = ver["on_2b"].fillna(0)
ver["on_1b"] = ver["on_1b"].fillna(0)

In [ ]:
ver["pitch_success"].value_counts()

In [ ]:
verClean = ver[['p_throws','balls','strikes','pitch_type', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','on_3b','on_2b','on_1b','outs_when_up','inning','release_pos_y','at_bat_number','pitch_number','pitch_name','bat_score','fld_score','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle', 'batter', 'pitch_success','pitch_outcome_score']]

In [ ]:
verClean = verClean.dropna()

In [ ]:
train, test = train_test_split(verClean, stratify=verClean["pitch_success"])
train

In [ ]:
xcols = ['p_throws','balls','strikes','pitch_type', 'release_speed', 'pfx_x', 'pfx_z', 'release_spin_rate','plate_x','plate_z','on_3b','on_2b','on_1b','outs_when_up','inning','release_pos_y','at_bat_number','pitch_number','pitch_name','bat_score','fld_score','pitcher_days_since_prev_game','n_thruorder_pitcher', 'arm_angle', 'batter']
ycol = "pitch_success"

In [ ]:
USE_PCA = True  # set True to activate

if USE_PCA:
    n_components = 20 # you can tune this
    p = Pipeline([
        ("oh", DataFrameEncoder()),
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=n_components)),
        ("lreg", LogisticRegression(max_iter = 10000, class_weight = {False:0.5,True:0.5}))
    ])
else:
    p = Pipeline([
        ("oh", DataFrameEncoder()),
        ('scaler', StandardScaler()),
        ("lreg", LogisticRegression(max_iter = 10000, class_weight = {False:0.5,True:0.5}))
    ])


In [ ]:
p.fit(train[xcols], train[ycol])

In [ ]:
test.copy()
test["predicted"] = p.predict(test[xcols])
test.head()

In [ ]:
actual = test[ycol]
predicted = test["predicted"]

In [ ]:
labels = [False,True]
cm = confusion_matrix(actual,predicted,labels)
cm = pd.DataFrame(cm, index=labels, columns=labels)
cm

In [ ]:
print("TP", cm.iloc[1,1]) # True Positives
print("TN", cm.iloc[0,0]) # True Negatives
print("FP", cm.iloc[0,1]) # False Positives
print("FN", cm.iloc[1,0]) # False Negatives

In [ ]:
accuracy_score(actual,predicted)